<a href="https://colab.research.google.com/github/AkyLast/AUTOMATIZACAO-Edeconsil/blob/main/FORMATTING%20AUTOMATION/Relat%C3%B3rios%20de%20Ocorr%C3%AAncias/FORA%20DE%20HOR%C3%81RIO%20(CRs)/Fora_do_Hor%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill, Font
from openpyxl.styles.borders import Border, Side

In [ ]:
FILE_NAME = "Relatorio_Formatado - FORAHORARIO.xlsx"

intervalo_time = "04-06 a 05-06"

FILA_SAVE_NAME = f"Relatório de Ocorrências Fora do Horário {intervalo_time}"

In [ ]:
df = pd.read_excel(FILE_NAME)

In [ ]:
df.loc[:, "CR"] = df["CR"].apply(
   lambda x: x.replace( " - ", " ").replace("  ", " ").replace("-", " ").strip().replace("ALUMAR REDUÇÃO", "ALUMAR OPERAÇÃO REDUÇÃO").replace("ALUMAR OPERAÇÃO ÁREA REDUÇÃO", "ALUMAR OPERAÇÃO REDUÇÃO").replace("NOVA SEDE VETOR", "NA LOGISTICA")
   )
df.loc[:, "CR"] = df["CR"].apply(lambda x: x.replace("EM MANUTENÇÃO OFICINA TRACBEL", "REGIONAL METROPOLITANA"))
CR = df['CR'].unique()

df.loc[:, "MOTORISTA"] = df["MOTORISTA"].apply(
    lambda x: x.replace("  ", " ").strip().replace("ALUMAR - OPERAÇÃO ÁREA REDUÇÃO", "ALUMAR OPERAÇÃO REDUÇÃO")
    )
df = df[["DATA", "HORA", "TAG", "MOTORISTA", "CR", "ENDEREÇO"]]
df["DATA"] = pd.to_datetime(df["DATA"], format="%d/%m/%Y")
df["DATA"] = df["DATA"].dt.strftime("%d/%m/%Y")

In [ ]:
# Função para formatar os dados antes de salvar
def formatar_e_salvar(df, arquivo, CR):
    for name_CR in CR:
        # Filtra o DataFrame com base no "CR"
        df_CR = df[df["CR"] == name_CR]

        df_CR = df_CR.sort_values(by='DATA', ascending=True)  # Ordena o DataFrame por data em ordem crescente

        # Adiciona uma coluna em branco antes de exportar
        df_CR.insert(0, '', '')  # Coluna em branco na posição 0

        # Salva o DataFrame no Excel sem o índice
        df_CR.to_excel(f"{arquivo} - {name_CR}.xlsx", index=False, engine='openpyxl')

        # Abre o arquivo Excel para aplicar as formatações
        wb = load_workbook(f"{arquivo} - {name_CR}.xlsx")
        ws = wb.active

        # Formatação do cabeçalho (linha 1) - fundo azul e texto branco
        header_fill = PatternFill(start_color="0000FF", end_color="0000FF", fill_type="solid")
        header_font = Font(color="FFFFFF", bold=True)  # Branco, Negrito

        # Começando da coluna 2 (segunda coluna) até a última coluna
        for col in range(2, len(df_CR.columns) + 1):  # A partir da segunda coluna
            cell = ws.cell(row=1, column=col)
            cell.fill = header_fill
            cell.font = header_font

        # Definindo as bordas
        border = Border(
            left=Side(border_style="thin", color="000000"),
            right=Side(border_style="thin", color="000000"),
            top=Side(border_style="thin", color="000000"),
            bottom=Side(border_style="thin", color="000000")
        )

        # Aplicando bordas nas células a partir da segunda coluna até a última
        for row in range(2, len(df_CR) + 2): # Por linhas
            for col in range(2, len(df_CR.columns) + 1):  # Por Colunas
                cell = ws.cell(row=row, column=col)
                cell.alignment = Alignment(horizontal='center', vertical='center')
                cell.border = border

        # Salva as alterações no arquivo Excel
        wb.save(f"{arquivo} - {name_CR}.xlsx")

# Chamada da função para formatar e salvar
formatar_e_salvar(df, FILA_SAVE_NAME, CR)
